<center> <h1> PSI3431 - Experiência 3 - Processos Estocásticos </h1> </center> <br> 
    
 Nome | NUSP 
 :--------------: | :--------------: 
 Bruno Mucha Pasini | 10845791 
 Pedro Henrique Gianjoppe dos Santos | 10774055 
 Caio Hudson Queiroz de Souza | 10773885 

In [1]:
 import Pkg; Pkg.add("Plots")
 import Pkg; Pkg.add("WAV")
 import Pkg; Pkg.add("DSP")
 import Pkg; Pkg.add("Statistics")
 import Pkg; Pkg.add("LaTeXStrings")
 import Pkg; Pkg.add("PlotlyJS")
 using Plots
 using Statistics
 using WAV
 using DSP
 using LaTeXStrings
plotlyjs();

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9530402813237408092\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-14063919087494124928\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9530402813237408092\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")